In [6]:
from sqlalchemy import create_engine

import re
import pandas as pd
import numpy as np
from plotnine import *
from pandas import DataFrame, read_csv
from wordcloud import WordCloud
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'wordcloud'

In [11]:
def stripChar(x):
    try:
        return re.sub("[()\"]","",x)
    except:
        return np.nan

In [ ]:
#MYSQL CONNECTIONS

mysql_user = "root"
mysql_password = "localhost"
mysql_database = "airline_passenger_satisfaction"
mysql_host = "127.0.0.1"
port = 3306

sqlEngine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_database}',pool_recycle=port)
dbConnection = sqlEngine.connect()

In [12]:
#AIRLINE REVIEWS

#Reading the Airline CSV files
def readData():
    colnames = ['airline_name','overall_score','review_title','reviewer_country','review_date','aircraft_model','travel_type','seat_type','route','date_flown','seat_comfort_rating','service_rating','food_rating','entertainment_rating','groundservice_rating','wifi_rating','value_rating','recommended']
    Afrance = pd.read_csv("/Users/skudli/Desktop/air-france.csv",header = None,names = colnames)
#     Ana = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/ana-all-nippon-airways.csv",header = None,names = colnames)
#     Emirates = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/emirates.csv",header = None,names = colnames)
#     Japan = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/japan-airlines.csv",header = None,names = colnames)
#     Korean = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/korean-air.csv",header = None,names = colnames)
#     Qantas = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/qantas-airways.csv",header = None,names = colnames)
#     Qatar = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/qatar-airways.csv",header = None,names = colnames)
#     Singapore = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/singapore-airlines.csv",header = None,names = colnames)
#     Swiss = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/swiss-international-air-lines.csv",header = None,names = colnames)
#     Turkish = pd.read_csv("/Users/shreyajaiswal/Desktop/Shreya/Semester_one/DMDD/Assignment 3 /Web_Scrapping_Data/Airlines_Data/turkish-airlines.csv",header = None,names = colnames)
#     df = pd.concat([Afrance,Afrance,Ana,Emirates,Japan,Korean,Qantas,Qatar,Singapore,Swiss,Turkish])
    df = pd.concat([Afrance])
    return df


In [13]:
#READ DATA
df_airline_reviews = readData()
df_airline_reviews

#DROP DUPLICATES
df_airline_reviews = df_airline_reviews.drop_duplicates(keep='first')

#DROP AIRLINE_REVIEWS ROW 0
df_airline_reviews = df_airline_reviews.drop([0])


#STRIP CHARACTERS
df_airline_reviews.review_title = df_airline_reviews.review_title.apply(lambda x: stripChar(x))
df_airline_reviews.reviewer_country = df_airline_reviews.reviewer_country.apply(lambda x: stripChar(x))
#display(df_airline_reviews)


#FILL EMPTY CELLS WITH VAL 0
df_airline_reviews.recommended = df_airline_reviews.recommended.fillna(value = 0)
#display(df_airline_reviews)

#FILL 1 TO CELLS WITH VAL YES
df_airline_reviews.recommended = df_airline_reviews.recommended.eq('yes').mul(1)
#display(df_airline_reviews)

#DATETIME CHANGES
df_airline_reviews['review_date'] = pd.to_datetime(df_airline_reviews['review_date'])
# df_airline_reviews['year'] = df_airline_reviews['review_date'].dt.year
# df_airline_reviews['month'] = df_airline_reviews['review_date'].dt.month
# df_airline_reviews.drop(['date_flown'], axis=1)


#display(df_airline_reviews)

In [17]:
#POPULATE AIRLINE TABLE

df_airline = df_airlines_merge[['iata_airline','icao_airline','airline_name','active']]
df_airline = df_airline.drop_duplicates(keep='first')
# df_airline


#df_airline=df_airline.drop_duplicates(subset = ['iata_airline'], keep = 'first', inplace = True) 
df_airline_final = df_airline


#WRITING THE TABLE TO DATABASE
df_airline_final.to_sql(con=dbConnection,name='AIRLINES',if_exists='append',index=False)

display(df_airline_final)
print("AIRLINES table added to database successfully")
len(list(df_country_airline['country_airline_id'].unique()))

NameError: name 'df_airlines_merge' is not defined

In [16]:
# df_new = df_airline_reviews[['airline_name']]
# #df_new['value']=df_airline_reviews.airline_name.str.lower().isin(list(df_airline_final.airline_name.str.lower()))
# df_new=df_new.loc[df_new['value'].isin([True])]
# df_new=df_new.drop_duplicates(keep='first')
# df_new

df_airline_final['airline_name'] = df_airline_final['airline_name'].str.lower()
df_airline_final

df_airlines_iata = df_airline_final[['airline_name','iata_airline']]
df_airlines_iata

#MERGING 
merge_df = pd.merge(df_airline_reviews, df_airlines_iata, on='airline_name', how='left')
df_review_main = merge_df
df_review_main

NameError: name 'df_airline_final' is not defined

In [ ]:
##POPULATE AIRLINE_REVIEWS TABLE

df_review_final = df_review_main[['iata_airline','review_title','review_date','seat_comfort_rating','service_rating','food_rating','entertainment_rating','groundservice_rating','wifi_rating','value_rating','overall_score','recommended']]
df_review_final = df_review_final.drop_duplicates(keep='first')
df_review_final


#WRITING THE TABLE TO DATABASE
df_review_final.to_sql(con=dbConnection,name='AIRLINE_REVIEWS',if_exists='replace',index=False)

display(df_country_airline)
print("AIRLINE_REVIEWS table added to database successfully")